## LLM

In [62]:
import os
import json
from dotenv import load_dotenv
from litellm import completion

from pydantic import BaseModel

# Model Config
load_dotenv()
config = {
    "model": f"azure/{os.getenv('AZURE_DEPLOYMENT')}",
    "api_key": os.getenv("AZURE_API_KEY"),
    "base_url": os.getenv("AZURE_ENDPOINT"),
    "api_version": os.getenv("AZURE_API_VERSION")
}

In [63]:
response = completion(
    **config,
    messages = [{"role": "user", "content": "good morning"}]
)

response.json()

{'id': 'chatcmpl-AyjMhOA0ZecwLt4FQdXv0OBPXxSt1',
 'created': 1739037427,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_f3927aa00d',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'Good morning! How can I assist you today?',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None,
    'refusal': None}}],
 'usage': {'completion_tokens': 10,
  'prompt_tokens': 9,
  'total_tokens': 19,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'service_tier': None,
 'prompt_filter_results': [{'prompt_index': 0,
   'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
    'jailbreak': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'sev

## Input Request

In [64]:
input_request = [{
    "feedback_id": "12345",
    "customer_name": "John Doe",
    "feedback_text": "The product is great, but the delivery was delayed.",
    "timestamp": "2025-01-10T10:30:00Z",
    "instructions": "Focus on identifying the sentiment and summarizing actionable insights."
},
{
    "feedback_id": "12345",
    "customer_name": "John Doe",
    "feedback_text": "hello how are you ??",
    "timestamp": "2025-01-10T10:30:00Z",
    "instructions": ""
}]

## Function Schema Genertion

In [65]:
def generate_function_schema(base_model: BaseModel):
    schema = base_model.model_json_schema()
    return {
        "type": "function",
        "function": {
            "name": schema.get("title", base_model.__name__),
            "description": schema.get("description", "None"),
            "parameters": {
                "type": schema.get("type", "object"),
                "properties": schema.get("properties")
            }
        }
    }


In [66]:
class SubAgent(BaseModel):
    """useful when to answer question based on feedback and Instructions.
    It can help with SentimentAnalysis, TopicCategorization, KeywordContextualization, Summarization
    """

In [67]:
schema = generate_function_schema(base_model=SubAgent)
schema

{'type': 'function',
 'function': {'name': 'SubAgent',
  'description': 'useful when to answer question based on feedback and Instructions.\nIt can help with SentimentAnalysis, TopicCategorization, KeywordContextualization, Summarization',
  'parameters': {'type': 'object', 'properties': {}}}}

## Step 1: SubAgent for Tool calling

## Sentiment Analysis Tool
- Perform sentiment scoring (positive, negative, neutral).

In [68]:
class SentimentAnalysisTool(BaseModel):
    query: str


def sentiment_analysis_tool(query: str, config: dict):
    # Prompt
    prompt = """Analyze the sentiment of the following user feedback and provide a JSON response with sentiment scores for positive, negative, and neutral categories. 
    
    NOTE: Ensure the sum of all values equals 1. Do not provide any explanation.  

    Output format:  
    ```json  
    {{ "positive": <score>, "negative": <score>, "neutral": <score>}}
    ```

    Feedback: {user_feedback}
    """.format(user_feedback=query)

    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ]
    )

    # Parsing and Loading
    output = response.choices[0].message.content.strip("```json").strip('```')[1:-1]
    return json.loads(output)

In [69]:
observation = sentiment_analysis_tool(
    query="The product is great, but the delivery was delayed", 
    config=config
)

print("Observation: ", observation)

Observation:  {'positive': 0.4, 'negative': 0.4, 'neutral': 0.2}


## Topic Categorization Tool
- Categorize feedback into predefined topics (e.g., Product Quality, Delivery, Support).



In [70]:
class TopicCategorizationTool(BaseModel):
    query: str

def topic_categorization_tool(query: str, config: dict):
    # Prompt
    prompt = """Categorize the following user feedback into one of the predefined topics: `Product Quality`, `Delivery`, `Support`.
    
    NOTE: Select only one category and assign it a confidence score between 0 and 1. 
    Do not provide any explanation.  

    Output format:  
    ```json  
    {{ "category": <Selected Category>, "score": <confidence_score>}}
    ```"  

    Feedback: {product_info}
    """.format(product_info=query)

    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ]
    )

    # Parsing and Loading
    output = response.choices[0].message.content.strip("```json").strip('```')[1:-1]
    return json.loads(output)

In [72]:
observation = topic_categorization_tool(
    query="The product is great, but the delivery was delayed", 
    config=config
)

print("Observation: ", observation)

Observation:  {'category': 'Delivery', 'score': 0.8}


## Keyword Contextualization Tool
- Extract context-aware keywords with relevance scores.

In [73]:
class KeywordContextualizationTool(BaseModel):
    query: str


def keyword_contextualization_tool(query: str, config: dict):
    # Prompt
    prompt = """Extract context-aware keywords from the following user feedback along with their relevance scores. 
    
    NOTE: Provide a JSON response where each keyword is mapped to a relevance score between 0 and 1. 
    Do not provide any explanation. 

    Output format:  
    ```json  
    {{ "keywords": {{ "<keyword1>": <score>, "<keyword2>": <score>, "<keyword3>": <score> }}}}
    ```"  

    Feedback: {user_feedback}
    """.format(user_feedback=query)

    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ]
    )

    # Parsing and Loading
    output = response.choices[0].message.content.strip("```json").strip('```')[1:-1]
    return json.loads(output)

In [74]:
observation = keyword_contextualization_tool(
    query="The product is great, but the delivery was delayed", 
    config=config
)

print("Observation: ", observation)

Observation:  {'keywords': {'product': 0.3, 'great': 0.2, 'delivery': 0.4, 'delayed': 0.6}}


## Summarization Tool
- Generate concise summaries and actionable recommendations.

In [75]:
class SummarizationTool(BaseModel):
    query: str


def summarization_tool(query: str, config: dict):
    # Prompt
    prompt = """Summarize the following user feedback concisely and provide actionable recommendations. 
    
    NOTE: Ensure the summary captures the core message, and the recommendations are practical and relevant. 
    Do not provide any explanation.  

    Output format:  
    ```json  
    {{  
    "summary": "<short concise summary>",  
    "recommendations": ["<short actionable recommendation 1>", "<short actionable recommendation 2>"]  
    }}  
    ```"  

    Feedback: {user_feedback}
    """.format(user_feedback=query)

    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ]
    )

    # Parsing and Loading
    output = response.choices[0].message.content.strip("```json").strip('```')[1:-1]
    return json.loads(output)

In [77]:
observation = summarization_tool(
    query="The product is great, but the delivery was delayed", 
    config=config
)

print("Observation: ", observation)

Observation:  {'summary': 'Users are satisfied with the product but are experiencing delivery delays.', 'recommendations': ['Improve delivery process to ensure timely shipments', 'Communicate proactively with customers about delivery status']}


## SubAgent Configuration 

In [78]:
tools = [SummarizationTool, SentimentAnalysisTool, KeywordContextualizationTool, TopicCategorizationTool]
sub_agent_tools = [generate_function_schema(base_model=tool) for tool in tools]

def sub_agent(input_request: dict, tools: list, config: dict):
    print(f"Initiate Sub agent : ...")
    prompt = """You are a specialized sub-agent equipped with four tools:  

        1. SentimentAnalysisTool – Analyzes sentiment (positive, negative, neutral) with confidence scores.  
        2. TopicCategorizationTool – Categorizes feedback into predefined topics (`Product Quality`, `Delivery`, `Support`).  
        3. KeywordContextualizationTool – Extracts context-aware keywords with relevance scores.  
        4. SummarizationTool – Generates concise summaries and actionable recommendations.  

    Your task is to answer the user based on feedback and instruction given.   
        - If an instruction is provided, select only the relevant tools accordingly.  
        - If no instruction is provided, use **all four tools** to extract comprehensive insights.  
        - You can use multiple tools at once when necessary.  

    #### **User Input:**  
        Feedback_text : {feedback_text}
        Instruction : {instructions}
    """.format(
        feedback_text=input_request.get("feedback_text", "N/A"),
        instructions=input_request.get("instructions", "N/A")
    )

    # LLm call
    print(f"Initiate Sub agent LLM call: ...")
    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ],
        tools=tools
    )
    
    print(f"Initiate Sub agent LLM call: {response}")
    
    # sub agent call
    if response.choices[0].message.tool_calls:
        result = tool_executor(
            input_request=input_request, 
            tool_calls=response.choices[0].message.tool_calls
        )
        return result["agent_response"]
    
    return response.choices[0].message.content

In [79]:
input_request[0]

{'feedback_id': '12345',
 'customer_name': 'John Doe',
 'feedback_text': 'The product is great, but the delivery was delayed.',
 'timestamp': '2025-01-10T10:30:00Z',
 'instructions': 'Focus on identifying the sentiment and summarizing actionable insights.'}

In [55]:
response = sub_agent(
    input_request=input_request[0], tools=sub_agent_tools, config=config
)

Action Name: SentimentAnalysisTool
Action Input: {'query': 'The product is great, but the delivery was delayed.'}
Action Result {'positive': 0.4, 'negative': 0.4, 'neutral': 0.2}
Action Name: SummarizationTool
Action Input: {'query': 'The product is great, but the delivery was delayed.'}
Action Result {'summary': 'Product quality is excellent, but delivery is slow.', 'recommendations': ['Improve delivery timelines', 'Communicate delay status promptly to customers']}


In [56]:
response

[{'sentimentanalysis': {'positive': 0.4, 'negative': 0.4, 'neutral': 0.2}},
 {'summarization': {'summary': 'Product quality is excellent, but delivery is slow.',
   'recommendations': ['Improve delivery timelines',
    'Communicate delay status promptly to customers']}}]

### Tool Executor 

In [83]:
class ToolExecutor:
    def __init__(self, tools: list, config: dict) -> None:
        # common tool executor
        self.config = config
        self.tools_by_name = {
            schema.get("title", "N/A"): tool for schema, tool in tools
        }

        # sub agent tools
        sub_agent_tools = [SummarizationTool, SentimentAnalysisTool, KeywordContextualizationTool, TopicCategorizationTool]
        self.sub_agent_tools = [generate_function_schema(base_model=tool) for tool in sub_agent_tools]

    def __call__(self, input_request: dict, tool_calls: list):
        output = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            print(f"Action Name: {tool_name}\nAction Input: {tool_args}")
            
            if tool_name != "SubAgent":
                tool_result = self.tools_by_name[tool_name](**tool_args, config=config)
            else:
                tool_result = self.tools_by_name[tool_name](
                    input_request=input_request,
                    tools = self.sub_agent_tools,
                    config=config
                    )
                
            print(f"Action Result: {tool_result}")
            output.append({f'{tool_name.lower().replace("tool","")}': tool_result})

        input_request["agent_response"] = output
        return input_request

executor_tools = [
    (SubAgent.model_json_schema(), sub_agent), 
    (SentimentAnalysisTool.model_json_schema(), sentiment_analysis_tool),
    (TopicCategorizationTool.model_json_schema(), topic_categorization_tool),
    (KeywordContextualizationTool.model_json_schema(), keyword_contextualization_tool),
    (SummarizationTool.model_json_schema(), summarization_tool)
]

tool_executor = ToolExecutor(tools=executor_tools, config=config)

##  Master Agent for Interaction

In [90]:
master_tools = [generate_function_schema(base_model=SubAgent)]

def master_agent(input_request: dict, tools: list, config: dict):
    print(f"Initiate User Request : {input_request}")
    # This is master agent Prompt
    prompt = """You are a master agent responsible for handling general user interactions, greetings, and generic inquiries. 
    However, when the user provides instructions or mentions product-related details, you must delegate the task to a specialized sub-agent.  

    NOTE:
    - If the input is a general inquiry, greeting, or small talk, respond directly.  
    - If the user input contains instructions or product-related details, do not respond directly. 
      Instead, forward the request to the sub-agent using the exact instruction and feedback text provided by the user. 
    - Ensure responses are clear and structured.

    This keeps the master agent focused on general conversations while offloading specific tasks to the sub-agent.

    USER_INPUTS:
        Feedback_text: {feedback_text}
        Instructions: {instructions}
    """.format(
        feedback_text=input_request.get("feedback_text", "N/A"),
        instructions=input_request.get("instructions", "N/A")
    )

    print("Triggered Master Agent: .....")
    # LLm call
    response = completion(
        **config, 
        messages=[
            {"role": "user", "content": prompt},
            ],
        tools=tools
    )

    print(f"Master Agent Response : {response.json()}")

    # sub agent call
    if response.choices[0].message.tool_calls:
        result = tool_executor(
            input_request=input_request, 
            tool_calls=response.choices[0].message.tool_calls
        )
        print(f"Master Agent Final Answer: {result}")
        return result
    input_request["agent_response"] = response.choices[0].message.content

    print(f"Master Agent Final Answer: {input_request}")
    return input_request

In [92]:
observation = master_agent(
    input_request=input_request[1], 
    config=config,
    tools=master_tools
)

Initiate User Request : {'feedback_id': '12345', 'customer_name': 'John Doe', 'feedback_text': 'hello how are you ??', 'timestamp': '2025-01-10T10:30:00Z', 'instructions': '', 'agent_response': "Hello! I'm here to assist you. I'm doing well, thank you for asking. How can I help you today?"}
Triggered Master Agent: .....
Master Agent Response : {'id': 'chatcmpl-AyjXcCiZXna3NGNHGVPNSPqgWIpiA', 'created': 1739038104, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'system_fingerprint': 'fp_f3927aa00d', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': "Hello! I'm doing well, thank you for asking. How can I assist you today?", 'role': 'assistant', 'tool_calls': None, 'function_call': None, 'refusal': None}}], 'usage': {'completion_tokens': 19, 'prompt_tokens': 221, 'total_tokens': 240, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_t

In [93]:
observation

{'feedback_id': '12345',
 'customer_name': 'John Doe',
 'feedback_text': 'hello how are you ??',
 'timestamp': '2025-01-10T10:30:00Z',
 'instructions': '',
 'agent_response': "Hello! I'm doing well, thank you for asking. How can I assist you today?"}

## Apply Guardrails

In [ ]:
import os
import json
from boto3 import client
from dotenv import load_dotenv

load_dotenv()

bedrock = client(
    "bedrock-runtime", 
    region_name="us-east-1"
)

In [19]:
response = bedrock.apply_guardrail(
    guardrailIdentifier="8j0uvjqavz1m",
    guardrailVersion="1",
    source="INPUT",
    content=[{"text": {"text": json.dumps({"test": "hello how are you"})}}]
)

In [20]:
response

{'ResponseMetadata': {'RequestId': 'f4c7f45b-f6b3-467c-a3d8-a98ababd41c6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 07:04:36 GMT',
   'content-type': 'application/json',
   'content-length': '615',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f4c7f45b-f6b3-467c-a3d8-a98ababd41c6'},
  'RetryAttempts': 0},
 'usage': {'topicPolicyUnits': 0,
  'contentPolicyUnits': 1,
  'wordPolicyUnits': 0,
  'sensitiveInformationPolicyUnits': 0,
  'sensitiveInformationPolicyFreeUnits': 0,
  'contextualGroundingPolicyUnits': 0},
 'action': 'NONE',
 'outputs': [],
 'assessments': [{'invocationMetrics': {'guardrailProcessingLatency': 206,
    'usage': {'topicPolicyUnits': 0,
     'contentPolicyUnits': 1,
     'wordPolicyUnits': 0,
     'sensitiveInformationPolicyUnits': 0,
     'sensitiveInformationPolicyFreeUnits': 0,
     'contextualGroundingPolicyUnits': 0},
    'guardrailCoverage': {'textCharacters': {'guarded': 29, 'total': 29}}}}],
 'guardrailCoverage': {'textChar

In [12]:
response = bedrock.apply_guardrail(
    guardrailIdentifier="8j0uvjqavz1m",
    guardrailVersion="1",
    source="INPUT",
    content=[{"text": {"text": json.dumps({"test": "I want to kill you"})}}]
)

{'ResponseMetadata': {'RequestId': '460b7ac9-9188-4205-a110-4b55161b226a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 09 Feb 2025 07:00:36 GMT',
   'content-type': 'application/json',
   'content-length': '1005',
   'connection': 'keep-alive',
   'x-amzn-requestid': '460b7ac9-9188-4205-a110-4b55161b226a'},
  'RetryAttempts': 0},
 'usage': {'topicPolicyUnits': 0,
  'contentPolicyUnits': 1,
  'wordPolicyUnits': 0,
  'sensitiveInformationPolicyUnits': 0,
  'sensitiveInformationPolicyFreeUnits': 0,
  'contextualGroundingPolicyUnits': 0},
 'action': 'GUARDRAIL_INTERVENED',
 'outputs': [{'text': 'Sorry, the model cannot answer this question.'}],
 'assessments': [{'contentPolicy': {'filters': [{'type': 'VIOLENCE',
      'confidence': 'MEDIUM',
      'filterStrength': 'HIGH',
      'action': 'BLOCKED'},
     {'type': 'HATE',
      'confidence': 'MEDIUM',
      'filterStrength': 'HIGH',
      'action': 'BLOCKED'}]},
   'invocationMetrics': {'guardrailProcessingLatency': 204,
    '

In [21]:
def parse_guardrail_response(response: dict) -> dict:
    """
    Parse the Bedrock guardrail response and return a structured explanation
    of why the content was blocked.
    """
    result = {
        "status": response.get("action", "UNKNOWN"),
        "is_blocked": False,
        "reasons": [],
        "details": {}
    }
    
    if response["action"] == "GUARDRAIL_INTERVENED":
        result["is_blocked"] = True
        
        if "assessments" in response:
            for assessment in response["assessments"]:
                if "contentPolicy" in assessment:
                    filters = assessment["contentPolicy"].get("filters", [])
                    
                    for filter_info in filters:
                        violation = {
                            "type": filter_info.get("type"),
                            "confidence": filter_info.get("confidence"),
                            "strength": filter_info.get("filterStrength"),
                            "action": filter_info.get("action")
                        }
                        result["reasons"].append(violation)
        
        if "guardrailCoverage" in response:
            result["details"]["coverage"] = response["guardrailCoverage"]
            
        if "usage" in response:
            result["details"]["usage"] = response["usage"]
    
    return result

response = parse_guardrail_response(response)

In [22]:
response

{'status': 'NONE', 'is_blocked': False, 'reasons': [], 'details': {}}

## Apply Chaching 

In [26]:
input_payload = {'feedback_id': '12345',
 'customer_name': 'John Doe',
 'feedback_text': 'hello how are you ??',
 'timestamp': '2025-01-10T10:30:00Z',
 'instructions': '',
 'agent_response': ["Hello! I'm doing well, thank you for asking. How can I assist you today?"]}

In [ ]:
import boto3
import json
import hashlib
from datetime import datetime, timedelta

def store_feedback_cache(feedback_id, cache_key, cached_result):
    TABLE_NAME = "FeedbackCache"

    dynamodb = boto3.client("dynamodb", region_name="us-east-1")
    
    last_updated = datetime.now().isoformat()
    
    dynamodb.put_item(
        TableName=TABLE_NAME,
        Item={
            "feedback_id": {"S": feedback_id},
            "cache_key": {"S": cache_key},
            "cached_result": {"S": json.dumps(cached_result)},
            "last_updated": {"S": last_updated}
        }
    )

In [ ]:
# Example Usage
feedback_id = input_payload["feedback_id"]
text = input_payload["feedback_text"] + " " + input_payload["instructions"]
cache_key = hashlib.sha256(text.encode()).hexdigest()
cache_data = input_payload["agent_response"]

save_feedback_cache(
    feedback_id=feedback_id,
    cache_key=cache_key,
    cached_result=cache_data
)
print("Cache stored.")

In [ ]:
def retrive_feedback_cache(feedback_id: str, cache_key=None):
    TABLE_NAME = "FeedbackCache"

    dynamodb = boto3.client("dynamodb", region_name="us-east-1")
    
    if cache_key:
        response = dynamodb.get_item(
            TableName=TABLE_NAME,
            Key={
                "feedback_id": {"S": feedback_id},
                "cache_key": {"S": cache_key}
            }
        )
        if "Item" in response:
            return {response["Item"]["cache_key"]["S"]:json.loads(response["Item"]["cached_result"]["S"])}  # Return cached result
        return None
    
    else:
        response = dynamodb.query(
            TableName=TABLE_NAME,
            KeyConditionExpression="feedback_id = :fid",
            ExpressionAttributeValues={":fid": {"S": feedback_id}}
        )
        output = {item["cache_key"]["S"]: json.loads(item["cached_result"]["S"]) for item in response.get("Items", [])}
        if output:
            return output
        else:
            return None

### Cache Hit

In [61]:
print(get_feedback_cache("12345"))

{'ef93596d7b46e990272cdf8118dee91c8a33f6b42fe3c9da1a404ee1dbc173c6': ["Hello! I'm doing well, thank you for asking. How can I assist you today?"]}


### Cache Miss

In [62]:
print(get_feedback_cache("12346"))

None


In [63]:
feedback_id = input_payload["feedback_id"]
text = input_payload["feedback_text"] + " " + input_payload["instructions"]
cache_key = hashlib.sha256(text.encode()).hexdigest()
cache_data = input_payload["agent_response"]

### Cache Hit

In [58]:
print(get_feedback_cache(feedback_id, cache_key))  

{'ef93596d7b46e990272cdf8118dee91c8a33f6b42fe3c9da1a404ee1dbc173c6': ["Hello! I'm doing well, thank you for asking. How can I assist you today?"]}


### Cache Miss

In [64]:
print(get_feedback_cache("23223", cache_key))

None


## Custom logger

In [ ]:
import logging
import boto3
import watchtower

client_logs = boto3.client(
    "logs", 
    region_name="us-east-1"
)

# Initialize CloudWatch Logs client
LOG_GROUP = "Expedite-Commerce-Feedback-Analysis"  
LOG_STREAM = "Expedite-Commerce-Feedback-Analysis-Stream"
 
handler = watchtower.CloudWatchLogHandler(
    log_group=LOG_GROUP, 
    stream_name=LOG_STREAM,
    boto3_client=client_logs
    )

formatter = logging.Formatter('%(asctime)s : %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Create the logging instance
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

In [73]:
logger.info("Hello, world! from local")

### Clear Cache

In [ ]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv()

TABLE_NAME = "FeedbackCache"

dynamodb = boto3.client("dynamodb", region_name="us-east-1")

In [22]:
items =dynamodb.scan(TableName=TABLE_NAME)

for item in items["Items"]:
    print("Deleting item: ", item)
    print("Deleting item: ", item["feedback_id"]["S"])
    dynamodb.delete_item(TableName=TABLE_NAME, Key={
        "feedback_id": {"S": str(item["feedback_id"]["S"])},
        "cache_key": {"S": str(item["cache_key"]["S"])}
        }
        )

Deleting item:  {'last_updated': {'S': '2025-02-09T19:02:54.743616'}, 'feedback_id': {'S': '15345'}, 'ttl': {'N': '1739108034'}, 'cached_result': {'S': '[{"sentimentanalysis": {"positive": 0.5, "negative": 0.3, "neutral": 0.2}}, {"summarization": {"summary": "The product is well-received, but delivery was delayed.", "recommendations": ["Improve delivery timelines.", "Communicate delivery status proactively to customers."]}}]'}, 'cache_key': {'S': '432988856b8ae00e00a51aef60b6f8f79d96266fbaf37a0ca7f62ab62a31d63e'}}
Deleting item:  15345


In [4]:
items =dynamodb.scan(TableName=TABLE_NAME)
items

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'F5Q24DHA8PSD6K061S0R8N45KBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 09 Feb 2025 17:03:53 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '39',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'F5Q24DHA8PSD6K061S0R8N45KBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3413411624'},
  'RetryAttempts': 0}}

In [5]:
items

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'F5Q24DHA8PSD6K061S0R8N45KBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 09 Feb 2025 17:03:53 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '39',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'F5Q24DHA8PSD6K061S0R8N45KBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3413411624'},
  'RetryAttempts': 0}}

In [18]:
from datetime import datetime
ttl = int(datetime.now().timestamp())
ttl

1739108076

In [32]:
items =dynamodb.scan(TableName=TABLE_NAME)
items

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': '3M09D1008DA0RG3R630KMSANH3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 09 Feb 2025 13:48:28 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '39',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3M09D1008DA0RG3R630KMSANH3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3413411624'},
  'RetryAttempts': 0}}

In [23]:
import time
time.time()

1739108382.5497642

In [6]:
response = dynamodb.describe_time_to_live(TableName=TABLE_NAME)
response

{'TimeToLiveDescription': {'TimeToLiveStatus': 'DISABLED'},
 'ResponseMetadata': {'RequestId': 'DVKACJBNPDU2Q6A1CB9QSJB5EBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 09 Feb 2025 17:04:05 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '57',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'DVKACJBNPDU2Q6A1CB9QSJB5EBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '243044618'},
  'RetryAttempts': 0}}

In [7]:
 # Enable TTL on an existing DynamoDB table
response = dynamodb.update_time_to_live(
    TableName=TABLE_NAME,
    TimeToLiveSpecification={
        'Enabled': True,
        'AttributeName': "ttl"
    }
)

In [8]:
response

{'TimeToLiveSpecification': {'Enabled': True, 'AttributeName': 'ttl'},
 'ResponseMetadata': {'RequestId': 'D8I4MR0VHU4SUNS8LJS2UDNHRRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 09 Feb 2025 17:04:08 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '66',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D8I4MR0VHU4SUNS8LJS2UDNHRRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2108816323'},
  'RetryAttempts': 0}}